## Introduction
TODO: Write some information for the problem definition + the data input and the desired output
Also include some basic information for the tool and how it works


## Case Study


### Load Data Input

In [1]:
from problem import CAOSProblem
import json

#Load Data
f = open("test_instance.json")
data = json.loads(f.read())
f.close()


### Create problem object and load data

In [2]:
p = CAOSProblem()
p.LoadInstance(data)

### Report the problem

In [3]:
p.Report()

#######################
### Instance Statistics
### Planning Horizon 5
### Total Contracts: 6
### Total CounterParties 4
#######################
### Inbound Contracts:
Period:  0  Client:  Amazon Amount:  1000
Period:  3  Client:  Tesla Amount:  2000
Period:  0  Client:  Google Amount:  3500
Outbound Contracts
Period:  0  Client:  Facebook Amount:  1000
Period:  1  Client:  Facebook Amount:  1000
Period:  2  Client:  Facebook Amount:  1500
#######################


### Solve

In [4]:
p.Solve()

Solving  2160 using 4 threads


d:\Projects\Consulting\AiPlan4EU\Code\lib\site-packages\unified_planning-0.4.2.458.dev1-py3.9.egg\unified_planning\io\pddl_writer.py:257: UserWarning: The PDDL printer cannot exactly represent the real constant '1080863910568919/36028797018963968'
  warn(


Threads Completed


KeyError: 0